In [38]:
import wandb
import pandas as pd
import plotly.express as px

api = wandb.Api()
runs = api.runs("lucyfarnik/jacobian_saes_jac_coef_sweep1")

metrics = ["losses/mse_loss", "metrics/explained_variance", "metrics/explained_variance_std", 
           "model_performance_preservation.ce_loss_score", "losses/jacobian_loss",
           "sparsity/mean_passes_since_fired", "sparsity/dead_features"]
data = []
for run in runs:
    run_data = { "jacobian_coefficient": run.config["jacobian_coefficient"] }
    for m in metrics:
        if "." in m:
            m0, m1 = m.split(".")
            run_data[m1] = run.summary[m0][m1]
        else:
            run_data[m.split("/")[1]] = run.summary[m]
    data.append(run_data)

df = pd.DataFrame(data)

In [42]:
jac_range = [0, 30_000]
df_filtered = df[df["jacobian_coefficient"] >= jac_range[0]]
df_filtered = df_filtered[df_filtered["jacobian_coefficient"] <= jac_range[1]]

for m in metrics:
    metric = m.split(".")[1] if "." in m else m.split("/")[1]
    px.scatter(df_filtered, x="jacobian_coefficient", y=metric, log_x=True, title=metric).show()